<div class="alert alert-info"><strong>Note:</strong> This notebook contains IPython widgets. Consequently, you won't be able to use Kernal/Restart &amp; Restart command to automatically execute all cells in the notebook. You must use Run command individually to execute each and advance to the next cell.</div>

Import Python modules...

In [ ]:
from __future__ import print_function

import os
import sys
import time
import re

import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import ipywidgets as widgets

from IPython.display import display, HTML
from IPython import __version__ as ipyVersion

# Import MW modules from the current directory or default Python directory...
import MWUtil

%matplotlib inline

print("Python: %s.%s.%s" % sys.version_info[:3])
print("IPython: %s" % ipyVersion)

print()
print(time.asctime())

The URL PATH

The MW REST URL consists of three main parts, separated by forward slashes, after the common prefix specifying the invariant base URL (https://www.metabolomicsworkbench.org/rest/):

https://www.metabolomicsworkbench.org/rest/context/input_specification/output_specification

Part 1: The context determines the type of data to be accessed from the Metabolomics Workbench, such as metadata or results related to the submitted studies, data from metabolites, genes/proteins and analytical chemistry databases as well as other services related to mass spectrometry and metabolite identification:

context = study | compound | refmet | gene | protein | moverz | exactmass

Part 2: The input specification consists of two required parameters describing the REST request:

input_specification = input_item/input_value

Part 3: The output specification consists of two parameters describing the output generated by the REST request:

output_specification = output_item/(output_format)

The first parameter is required in most cases. The second parameter is optional. The input and output specifications are context sensitive. The context determines the values allowed for the remaining parameters in the input and output specifications as detailed in the sections below.

Setup MW REST base URL...

In [ ]:
MWBaseURL = "https://www.metabolomicsworkbench.org/rest"

**Retrieve and process results data for named metabolities...**

Retrieve named metabolities data for a studies...

In [ ]:
# Initialize data...
StudiesResultsData = None

In [ ]:
# Setup UIF to retrieve data...
StudyIDText = widgets.Text(value = "ST000001", description = "Study ID (s)", placeholder = "Type study ID", disabled = False)

RetrieveDataBtn = widgets.Button(description = 'Retrieve Data', disabled = False, button_stype = '',
                                 tooltip = "Retrieve data for study ID")
OutputRetrieveDataBtn = widgets.Output()


WarningTextHTML = widgets.HTML(value="<div class='alert alert-warning'><strong>Warning:</strong> Don't re-run the current cell after specifying study ID(s) and retrieving the data. Click on the next cell to advance.</div>", placeholder='', description='')

def RetrieveAndListData(Object):
    global StudiesResultsData
    
    StudyIDs = StudyIDText.value
    
    OutputRetrieveDataBtn.clear_output()
    with OutputRetrieveDataBtn:
        if len(StudyIDs):
            print("\nProcessing study ID(s): %s" % StudyIDs)
            StudiesResultsData = MWUtil.RetrieveStudiesAnalysisAndResultsData(StudyIDs, MWBaseURL)
            DisplayData = False if len(StudiesResultsData.keys()) > 1 else True
            MWUtil.ListStudiesAnalysisAndResultsData(StudiesResultsData, DisplayDataFrame = DisplayData,
                                              IPythonDisplayFuncRef = display, IPythonHTMLFuncRef = HTML)
        else:
            print("\nNo study ID(s) specified...")

RetrieveDataBtn.on_click(RetrieveAndListData)

display(StudyIDText, RetrieveDataBtn, WarningTextHTML)
display(OutputRetrieveDataBtn)

In [ ]:
if StudiesResultsData is None or len(StudiesResultsData.keys()) == 0:
    print("Failed to retrieve data. Specify alid study ID(s) and click button above without re-running the cell...")
else:
    print("Successfully retrieved data for specified study ID(s) %s..." % StudyIDText.value)

Setup UIF for selecting and plotting available data...

In [ ]:
# Setup UIF data...
StudiesUIFData = MWUtil.SetupUIFDataForStudiesAnalysisAndResults(StudiesResultsData, MinClassCount = 2)

In [ ]:
if StudiesUIFData is None or len(StudiesUIFData.keys()) == 0:
    print("Failed to retrieve data containing multiple classes. Specify valid study ID(s) and try again...")
else:
    print("Successfully retrieved data for specified study ID(s) %s..." % StudyIDText.value)

In [ ]:
# Setup a function to perform PCA and generate dataframe for PCA plot...
def GeneratePCAPlotData(InputDataFrame, NumComponents = 2, ClassColID = "Class", ClassNumColID = "ClassNum"):
    """Perform PCA and generate plot data frame. """
    
    DataFrame = InputDataFrame.copy()
    
    # Drop Class column...
    if ClassColID is not None:
        DataFrame = DataFrame.drop(ClassColID, axis = 1)
    
    # Setup a class dataframe...
    ClassesDataFrame = DataFrame.loc[:,[ClassNumColID]]
    
    # Setup a features dataframe for metaboloties...
    FeaturesDataFrame = DataFrame.drop(ClassNumColID, axis = 1)
    
    # Standardize the featues...
    FeaturesDataValues = FeaturesDataFrame.values
    FeaturesDataValues = StandardScaler().fit_transform(FeaturesDataValues)
    
    # Perform analysis...
    PCAModel = PCA(n_components = NumComponents)
    PrincipalComponents = PCAModel.fit_transform(FeaturesDataValues)
    PrincipalComponents = PrincipalComponents.tolist()
    
    PCAColumnNames = []
    for Index in range(len(PrincipalComponents[0])):
        ColumnID = "PC%s" % (Index + 1)
        PCAColumnNames.append(ColumnID)
        
    IndexValues = ClassesDataFrame.index.values.tolist()
    PCADataFrame = pd.DataFrame(data = PrincipalComponents, columns = PCAColumnNames, index = IndexValues)
    
    # Setup final PCA dataframe including class numbers...
    PCADataFrame = pd.concat([ClassesDataFrame, PCADataFrame], axis = 1)
    
    ExplainedVariance = PCAModel.explained_variance_ratio_
    ExplainedVariance = ExplainedVariance.tolist()
    
    return (PCADataFrame, ExplainedVariance)

# Setup a function to draw PCA plot...
def DrawPCAPlot(PCAPlotDataFrame, ClassNumColID ="ClassNum", PC1ColID = "PC1", PC2ColID = "PC2",
                ColorPaletteName = "bright", PlotStyle = "darkgrid", FontScale = 1.3,
                TitleFontWeight = "bold", LabelsFontWeight = "bold",
                PlotWidth = 9, PlotHeight = 6):
    
    sns.set(rc = {'figure.figsize':(PlotWidth, PlotHeight)})
    sns.set(style = PlotStyle, font_scale = FontScale)
    
    # Color palette names: deep, muted, pastel, bright, dark, and colorblind.
    NumOfClasses = len(PCAPlotDataFrame[ClassNumColID].unique().tolist())
    ColorsPalette = sns.color_palette(ColorPaletteName, NumOfClasses)
    
    StyleColID = ClassNumColID if NumOfClasses <= 5 else None
    if StyleColID is not None:
        Axis = sns.scatterplot(x = PC1ColID, y = PC2ColID, hue = ClassNumColID, style = StyleColID,
                               data = PCAPlotDataFrame, palette = ColorsPalette, legend = "brief")
    else:
        Axis = sns.scatterplot(x = PC1ColID, y = PC2ColID, hue = ClassNumColID, data = PCAPlotDataFrame,
                               palette = ColorsPalette, legend = "brief")
    
    # Set title and labels...
    Axis.set_title("PCA Scores Plot", fontweight = TitleFontWeight)
    Axis.set_xlabel(PC1ColID, fontweight = LabelsFontWeight)
    Axis.set_ylabel(PC2ColID, fontweight = LabelsFontWeight)
    
    # Draw legend outside the plot...
    plt.legend(bbox_to_anchor = (1.05, 1), loc = 2, borderaxespad = 0.)
    
    plt.show()

In [ ]:
# Setup UIF...
FirstStudyID = StudiesUIFData["StudyIDs"][0]
StudiesDropdown = widgets.Dropdown(options = StudiesUIFData["StudyIDs"], value = FirstStudyID, 
                                   description="Study:", disabled = False)

FirstAnalysisID = StudiesUIFData["AnalysisIDs"][FirstStudyID][0]
AnalysisDropdown = widgets.Dropdown(options = StudiesUIFData["AnalysisIDs"][FirstStudyID], value = FirstAnalysisID,
                                    description = "Analysis:", disabled = False)

Components = ["2", "3", "4", "5"]
ComponentsDropdown = widgets.Dropdown(options = Components, value = "2", description = "Components:")

PlotStyles = ["Darkgrid", "Whitegrid", "Dark", "White", "Ticks"]
PlotStylesDropdown = widgets.Dropdown(options = PlotStyles, value = "Darkgrid", description = "Plot style:")

DefaultPlotWidth = 9
DefaultPlotHeight = 6
PlotSizeText = widgets.Text(value = "9x6", description = "Plot size:", placeholder = "Type WxH; Hit enter",
                            disabled = False, continuous_update=False)

PlotColorPalettes = ["Deep", "Muted", "Pastel", "Bright", "Dark", "Colorblind"]
PlotColorPalettesDropdown = widgets.Dropdown(options = PlotColorPalettes, value = "Bright", description = "Color palette:")


DataLayout = widgets.Layout(margin='0 0 4px 0')
StudiesDataHBox = widgets.HBox([StudiesDropdown, AnalysisDropdown], layout = DataLayout)
ComponentsDataHBox = widgets.HBox([ComponentsDropdown], layout = DataLayout)
PlotsDataHBox = widgets.HBox([PlotStylesDropdown, PlotSizeText], layout = DataLayout)
ColorPalattesDataHBox = widgets.HBox([PlotColorPalettesDropdown], layout = DataLayout)

Output = widgets.Output()
OutputPlot = widgets.Output()

UpdatePlot = True

def DisablePlotUpdate():
    global UpdatePlot
    UpdatePlot = False

def EnablePlotUpdate():
    global UpdatePlot
    UpdatePlot = True

def GetUpdatePlotStatus():
    global UpdatePlot
    return True if UpdatePlot else False

# Setup function to update dropdown options...
def UpdateAnalysisDropdown(StudyID):
    AnalysisDropdown.options = StudiesUIFData["AnalysisIDs"][StudyID]
    AnalysisDropdown.value = StudiesUIFData["AnalysisIDs"][StudyID][0]

# Setup dropdown event handlers...
def StudiesDropdownEventHandler(Change):
    StudyID = Change["new"]
    
    DisablePlotUpdate()
    UpdateAnalysisDropdown(StudyID)
    EnablePlotUpdate()
    
    PlotData()

def AnalysisDropdownEventHandler(Change):
    PlotData()

def ComponentsDropdownEventHandler(Change):
    PlotData()

def PlotStylesDropdownEventHandler(Change):
    PlotData()
    
def PlotSizeTextEventHandler(Change):
    PlotData()
    
def PlotColorPalettesDropdownEventHandler(Change):
    PlotData()
    
# Bind required event handlers...
StudiesDropdown.observe(StudiesDropdownEventHandler, names = 'value')
AnalysisDropdown.observe(AnalysisDropdownEventHandler, names = 'value')
ComponentsDropdown.observe(ComponentsDropdownEventHandler, names = 'value')

PlotStylesDropdown.observe(PlotStylesDropdownEventHandler, names = 'value')
PlotSizeText.observe(PlotSizeTextEventHandler, names = 'value')
PlotColorPalettesDropdown.observe(PlotColorPalettesDropdownEventHandler, names = 'value')

# List class information...
def ListClassInformation(StudyID, AnalysisID):
    """List information for available class names and numbers."""
    print("StudyID: %s" % (StudyID))
    print("AnalysisID: %s\nAnalysis Summary: %s" % (AnalysisID, StudiesResultsData[StudyID][AnalysisID]["analysis_summary"]))
    for ClassName in StudiesResultsData[StudyID][AnalysisID]["class_names_to_nums"]:
        ClassNum = StudiesResultsData[StudyID][AnalysisID]["class_names_to_nums"][ClassName]
        print("ClassNum: %s; ClassName: %s" % (ClassNum, ClassName))
    
# Set up function to generate PCA plot...
def PlotData():
    if not UpdatePlot:
        return
    
    Output.clear_output()
    OutputPlot.clear_output()
    
    StudyID = StudiesDropdown.value
    AnalysisID = AnalysisDropdown.value
    DataFrame = StudiesResultsData[StudyID][AnalysisID]["data_frame"]
    
    NumOfComponents = int(ComponentsDropdown.value)
    
    Style = PlotStylesDropdown.value
    Style = Style.lower()
    
    Palette = PlotColorPalettesDropdown.value
    Palette = Palette.lower()
    
    PlotSize = PlotSizeText.value.lower()
    PlotSize = re.sub(" ", "", PlotSize)
    PlotSizeWords = PlotSize.split("x")
    
    if len(PlotSizeWords) == 2 and len(PlotSizeWords[0]) > 0 and len(PlotSizeWords[1]) > 0:
        Width = float(PlotSizeWords[0])
        Height = float(PlotSizeWords[1])
    else:
        Width = DefaultPlotWidth
        Height = DefaultHeight
        with Output:
            print("Invalid plot size; Using default plot size: %sx%s\n" % (Width, Height))
    
    # Retrieve data for a PCA plot...
    with OutputPlot:
        PCAPlotDataFrame, ExplainedVariance = GeneratePCAPlotData(DataFrame, NumComponents = NumOfComponents,
                                                                  ClassColID = "Class", ClassNumColID = "ClassNum")
    with OutputPlot:
        # Draw PCA plot...
        DrawPCAPlot(PCAPlotDataFrame, ClassNumColID ="ClassNum", PC1ColID = "PC1", PC2ColID = "PC2",
                ColorPaletteName = Palette, PlotStyle = Style,
                PlotWidth = Width, PlotHeight = Height)
        
        print("Explained variance:")
        for Index in range(len(ExplainedVariance)):
            ID = "PC%s" % (Index + 1)
            print("%s: %.2f%s" % (ID, (ExplainedVariance[Index] * 100), "%"))
    
    with Output:
        ListClassInformation(StudyID, AnalysisID)
        
        # Setup a link to download ata...
        FileName = "%s_%s_Data.csv" % (StudyID, AnalysisID)
        HTMLText = MWUtil.SetupCSVDownloadLink(DataFrame, Title = "Download data", CSVFilename = FileName)
        display(HTML(HTMLText))
        
        # Setup a link to download PCA data...
        FileName = "%s_%s_PCA_Data.csv" % (StudyID, AnalysisID)
        HTMLText = MWUtil.SetupCSVDownloadLink(PCAPlotDataFrame, Title = "Download PCA data", CSVFilename = FileName)
        display(HTML(HTMLText))
    
        # List dataframe...
        display(HTML(PCAPlotDataFrame.to_html(max_rows = 10, max_cols = 10)))


display(StudiesDataHBox)
display(ComponentsDataHBox)
display(PlotsDataHBox)
display(ColorPalattesDataHBox)

display(OutputPlot)
display(Output)

PlotData()
